In [1]:
# Setting up the Coordinate Reference Systems up front in the necessary format.
crs_degree = {'init': 'epsg:4326'} # CGS_WGS_1984 (what the GPS uses)

# --- Paths

# Root path of Fremont Dropbox
import os
import sys
# We let this notebook to know where to look for fremontdropbox module
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from fremontdropbox import get_dropbox_location
# Root path of the Dropbox business account



dbx = get_dropbox_location()

# Temporary! Location of the folder where the restructuring is currently happening
data_path = dbx + '/Private Structured data collection'

aux_files = data_path+'/Data processing/Auxiliary files'

# Processing output path
output_path = aux_files + '/OD demand'

In [2]:
# Read more about GeoPandas data structures here: http://geopandas.org/data_structures.html
from geopandas import GeoDataFrame
import geopandas as gpd # To work with spatial data in a DataFrame
from shapely.geometry import Point, LineString # To create line geometries that can be used in a GeoDataFrame
from keplergl import KeplerGl
import pandas as pd

# packages for network manipulation and visualization
import csv
from operator import itemgetter
import networkx as nx
import osmnx as ox
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
#import cartopy.crs as ccrs
from IPython.display import Image
#from mpl_toolkits.basemap import Basemap as Basemap #Installation failed
from networkx.algorithms import community #This part of networkx, for community detection, needs to be imported separately.


#pandana
import pandana as pdna
from pandana.loaders import osm
%matplotlib inline
from descartes import PolygonPatch
import random
import warnings
warnings.filterwarnings('ignore')

#change the projection of the code
import pyproj as proj

### Load the Aimsun network using:
- Geopandas (to read and convert the files) 
- Networkx (to convert the data to a network) 
- Visualize the Geopandas network shapefile in Kepler.gl

In [3]:
#centroid_connections = GeoDataFrame.from_file(data_path + "/Data processing/Raw/Network/Aimsun/centroid_connections.shp")
#centroids = GeoDataFrame.from_file(data_path + "/Data processing/Raw/Network/Aimsun/centroids.shp")
#detectors = GeoDataFrame.from_file(data_path + "/Data processing/Raw/Network/Aimsun/detectors.shp")
#meterings = GeoDataFrame.from_file(data_path + "/Data processing/Raw/Network/Aimsun/meterings.shp")
nodes = GeoDataFrame.from_file(data_path + "/Data processing/Raw/Network/Aimsun/nodes.shp")
#polygons = GeoDataFrame.from_file(data_path + "/Data processing/Raw/Network/Aimsun/polygons.shp")
sections = GeoDataFrame.from_file(data_path + "/Data processing/Raw/Network/Aimsun/sections.shp")
#sectionsGeo = GeoDataFrame.from_file(data_path + "/Data processing/Raw/Network/Aimsun/sectionsGeo.shp")


In [4]:
Aimsun_nodes = nodes.to_crs(epsg=4326)
Aimsun_sections = sections.to_crs(epsg=4326)

In [5]:
Aimsun_sections.head(5)

,id,eid,name,nb_lanes,speed,capacity,rd_type,func_class,fnode,tnode,geometry
0,242.0,242,None,1,120.0,2100.0,175.0,1,9845.0,9923.0,"LINESTRING (-121.92244 37.49593, -121.92242 37..."
1,243.0,243,None,3,104.0,6300.0,175.0,1,9852.0,9848.0,"LINESTRING (-121.92313 37.49526, -121.92173 37..."
2,244.0,244,None,3,104.0,6300.0,175.0,1,9850.0,9852.0,"LINESTRING (-121.92352 37.49561, -121.92313 37..."
3,246.0,246,Geyser Court,1,50.0,700.0,179.0,5,9868.0,NaN,"LINESTRING (-121.91449 37.50455, -121.91477 37..."
4,247.0,247,Geyser Court,1,50.0,700.0,179.0,5,NaN,9868.0,"LINESTRING (-121.91572 37.50422, -121.91560 37..."


In [6]:
map_1 = KeplerGl(height=1000)
map_1.add_data(data=Aimsun_sections, name = "sections")
map_1.add_data(data=Aimsun_nodes, name = "nodes")
map_1



User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md


KeplerGl(data={'sections': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20…

In [7]:
# Loading the Aimsun network in networkx
# Nodes = nx.read_shp(data_path + "/Data processing/Raw/Network/Aimsun/nodes.shp")
# Edges = nx.read_shp(data_path + "/Data processing/Raw/Network/Aimsun/sections.shp")
#Point of Interests scraped from Google API
POIs = pd.read_csv(data_path + "/Data processing/Raw/Network/KPIs/locations_crawl.csv")
POIs['geometry'] = [Point(xy) for xy in zip(POIs['Long'], POIs['Lat'])]
crs = {'init': 'epsg:4326'}
# Creating a Geographic data frame for Point of Interests
POIs_gdf = gpd.GeoDataFrame(POIs, crs=crs)

In [8]:
POIs_gdf.head(5)

,Unnamed: 0,Names,Types,Lat,Long,geometry
0,0,Fremont,"['locality', 'political']",37.548540,-121.988583,POINT (-121.98858 37.54854)
1,1,ProCreativeWriters,"['point_of_interest', 'establishment']",37.504377,-121.964423,POINT (-121.96442 37.50438)
2,2,Baylands,"['neighborhood', 'political']",37.485034,-121.964375,POINT (-121.96437 37.48503)
3,3,Fremont,"['locality', 'political']",37.548540,-121.988583,POINT (-121.98858 37.54854)
4,4,Mission Sierra Taekwon-Do,"['health', 'point_of_interest', 'establishment']",37.532586,-121.922044,POINT (-121.92204 37.53259)


## Using Pandana package to plot the accessibility heatmap

preprocess the dataset before dumping the network into pandana

In [10]:
Aimsun_nodes['x'] = Aimsun_nodes['geometry'].x
Aimsun_nodes['y'] = Aimsun_nodes['geometry'].y

In [11]:
Aimsun_nodes = Aimsun_nodes.rename(columns={'id': "id_node"})
Aimsun_nodes['id_node'] = Aimsun_nodes['id_node'].astype(int)
Aimsun_nodes.index = Aimsun_nodes['id_node']
Aimsun_nodes_gdf = Aimsun_nodes[['id_node', 'x', 'y']]

In [12]:
Aimsun_nodes_gdf.head(5)

,id_node,x,y
id_node,,,
9845,9845,-121.922491,37.495935
9848,9848,-121.921727,37.494013
9850,9850,-121.923525,37.495613
9852,9852,-121.923128,37.495265
9854,9854,-121.934705,37.505858


In [13]:
Aimsun_sections.index = range(len(Aimsun_sections))
Aimsun_sections_gdf = Aimsun_sections[['id', 'fnode', 'tnode', 'capacity']]
Aimsun_sections_gdf['id'] = Aimsun_sections_gdf['id'].fillna(0.0).astype(int)

In [14]:
Aimsun_sections['fnode'].fillna(9845, inplace=True)
Aimsun_sections['tnode'].fillna(9845, inplace=True)
Aimsun_sections.index = range(len(Aimsun_sections))
Aimsun_sections_gdf = Aimsun_sections[['id', 'eid', 'fnode', 'tnode', 'capacity']]
Aimsun_sections_gdf['id'] = Aimsun_sections_gdf['id'].fillna(0.0).astype(int)

Aimsun_sections_gdf['tnode'] = Aimsun_sections_gdf['tnode'].astype(int)
Aimsun_sections_gdf['fnode'] = Aimsun_sections_gdf['fnode'].astype(int)
Aimsun_sections_gdf.id = range(len(Aimsun_sections_gdf))

In [15]:
Aimsun_sections.head(5)

,id,eid,name,nb_lanes,speed,capacity,rd_type,func_class,fnode,tnode,geometry
0,242.0,242,None,1,120.0,2100.0,175.0,1,9845.0,9923.0,"LINESTRING (-121.92244 37.49593, -121.92242 37..."
1,243.0,243,None,3,104.0,6300.0,175.0,1,9852.0,9848.0,"LINESTRING (-121.92313 37.49526, -121.92173 37..."
2,244.0,244,None,3,104.0,6300.0,175.0,1,9850.0,9852.0,"LINESTRING (-121.92352 37.49561, -121.92313 37..."
3,246.0,246,Geyser Court,1,50.0,700.0,179.0,5,9868.0,9845.0,"LINESTRING (-121.91449 37.50455, -121.91477 37..."
4,247.0,247,Geyser Court,1,50.0,700.0,179.0,5,9845.0,9868.0,"LINESTRING (-121.91572 37.50422, -121.91560 37..."


In [16]:
Aimsun_sections_gdf.head(5)

,id,eid,fnode,tnode,capacity
0,0,242,9845,9923,2100.0
1,1,243,9852,9848,6300.0
2,2,244,9850,9852,6300.0
3,3,246,9868,9845,700.0
4,4,247,9845,9868,700.0


In [17]:
id_keys = range(len(Aimsun_nodes_gdf))
id_values = Aimsun_nodes_gdf.id_node
id_dictionary = dict(zip(id_keys, id_values))

x_values = Aimsun_nodes_gdf.x
x_dictionary = dict(zip(id_keys, x_values))

y_values = Aimsun_nodes_gdf.y
y_dictionary = dict(zip(id_keys, y_values))

edge_keys = range(len(Aimsun_sections_gdf))
edge_values = Aimsun_sections.eid.astype(int)
edge_dictionary = dict(zip(edge_keys, edge_values))

from_values = Aimsun_sections_gdf.fnode.fillna(0.0).astype(int)
to_values = Aimsun_sections_gdf.tnode.fillna(0.0).astype(int)

from_dict = dict(zip(edge_keys, from_values))
to_dict = dict(zip(edge_keys, to_values))

weight_values = Aimsun_sections_gdf.capacity
weight_dict = dict(zip(edge_keys, weight_values))

In [18]:
#check to ensure that all the section ends are network nodes
node_set = set(Aimsun_nodes_gdf.id_node)
for node in Aimsun_sections.fnode.fillna(0.0).astype(int):
    if node not in node_set:
        print(node)

In [19]:
Aimsun_edge_dict = {
 'id': edge_dictionary,
 'id_node_source': from_dict,
 'id_node_target': to_dict,
 'distance': weight_dict}

node_dict = {
 'id_node': id_dictionary,
 'x': x_dictionary,
 'y': y_dictionary}

# read dictionary into dataframe
edges_topo = pd.DataFrame.from_dict(Aimsun_edge_dict)
nodes_gdf = pd.DataFrame.from_dict(node_dict)
nodes_gdf.index = nodes_gdf['id_node']



## Loading the edge weights into the Pandana network sections

In [20]:
vehSectTraj = pd.read_csv(data_path + '/Aimsun/Outputs/vehSectTrajectory.csv')
vehSectTraj = vehSectTraj[vehSectTraj.notna()]

In [21]:
vehSectTraj_temp = vehSectTraj.groupby("sectionId").first()

In [22]:
edge_weighted = pd.merge(edges_topo,
                           vehSectTraj_temp,
                           left_on='id',
                           right_on='sectionId',
                           how='left',
                           sort=True)

In [23]:
edge_weighted = edge_weighted[edge_weighted.travelTime.notna()]

In [24]:
#initialize the Pandana network 
net = pdna.Network(node_x = nodes_gdf["x"],
                   node_y = nodes_gdf["y"],
                   edge_from = edge_weighted["id_node_source"], 
                   edge_to = edge_weighted["id_node_target"],
                   edge_weights = edge_weighted[["travelTime"]])

In [25]:
#scraped Point of Interests in Fremont Area
POIs_gdf.head()

,Unnamed: 0,Names,Types,Lat,Long,geometry
0,0,Fremont,"['locality', 'political']",37.548540,-121.988583,POINT (-121.98858 37.54854)
1,1,ProCreativeWriters,"['point_of_interest', 'establishment']",37.504377,-121.964423,POINT (-121.96442 37.50438)
2,2,Baylands,"['neighborhood', 'political']",37.485034,-121.964375,POINT (-121.96437 37.48503)
3,3,Fremont,"['locality', 'political']",37.548540,-121.988583,POINT (-121.98858 37.54854)
4,4,Mission Sierra Taekwon-Do,"['health', 'point_of_interest', 'establishment']",37.532586,-121.922044,POINT (-121.92204 37.53259)


In [26]:
#only keep the POIs within Aimsun network
bounding_box = Aimsun_sections.unary_union.envelope
tem_df = gpd.GeoDataFrame(gpd.GeoSeries(bounding_box), columns=['geometry'])
intersections = gpd.overlay(POIs_gdf, tem_df, how='intersection')

In [27]:
#save the scraped Point of Interests to shapefile
intersections.to_file(driver = 'ESRI Shapefile', filename= data_path + "/Data processing/Raw/Network/Aimsun/POI.shp")

In [28]:
#visualize the POIs 
map_2 = KeplerGl(height=1000)
map_2.add_data(data=intersections, name = "Point Of Interests")
map_2.add_data(data=Aimsun_sections, name = "sections")
map_2.add_data(data=Aimsun_nodes, name = "nodes")
map_2




User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md


KeplerGl(data={'Point Of Interests': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, …

## Assign each POI to corresponding nearest Aimsun node

In [29]:
#get_node_ids uses the KDTree from scipy
near_ids = net.get_node_ids(intersections['Long'],
                            intersections['Lat'])

# Set the response as a new column on the POI reference df
intersections['nearest_node_id'] = near_ids

In [30]:
nodes_gdf.index = range(len(nodes_gdf))

In [31]:
# Create a merged dataframe that holds the node data (esp. x and y values)
# that relate to each nearest neighbor of each POI
nearest_to_pois = pd.merge(intersections,
                           nodes_gdf,
                           left_on='nearest_node_id',
                           right_on='id_node',
                           how='left',
                           sort=False,
                           suffixes=['_from', '_to'])

In [32]:
#showing how each POI is matched to the nearest node id
nearest_to_pois.head(5)

,Unnamed: 0,Names,Types,Lat,Long,geometry,nearest_node_id,id_node,x,y
0,4,Mission Sierra Taekwon-Do,"['health', 'point_of_interest', 'establishment']",37.532586,-121.922044,POINT (-121.92204 37.53259),26626,26626,-121.921920,37.532924
1,5,Hoard Siu PC,"['doctor', 'health', 'point_of_interest', 'est...",37.532591,-121.922271,POINT (-121.92227 37.53259),26645,26645,-121.922656,37.532504
2,6,Tavares Realty,"['real_estate_agency', 'point_of_interest', 'e...",37.533002,-121.922212,POINT (-121.92221 37.53300),23422,23422,-121.922149,37.533130
3,7,Unifier Learning Academy,"['point_of_interest', 'establishment']",37.532530,-121.922021,POINT (-121.92202 37.53253),26626,26626,-121.921920,37.532924
4,8,Oroysom Village,"['point_of_interest', 'establishment']",37.533912,-121.923178,POINT (-121.92318 37.53391),16578,16578,-121.923020,37.534326


In [33]:
POI_matching_lines = []
for row_id, row in nearest_to_pois.iterrows():
    linestr = LineString([(row['geometry'].x, row['geometry'].y),
                          (row['x'], row['y'])]).buffer(0.000001)
    POI_matching_lines.append(linestr)
    

In [34]:
POI_matching_lines_gdf = pd.DataFrame({'geometry':POI_matching_lines})
POI_matching_lines_gdf = gpd.GeoDataFrame(POI_matching_lines_gdf, crs=crs)

In [35]:
POI_matching_lines_gdf.head(5)

,geometry
0,"POLYGON ((-121.92192 37.53292, -121.92192 37.5..."
1,"POLYGON ((-121.92266 37.53250, -121.92266 37.5..."
2,"POLYGON ((-121.92215 37.53313, -121.92215 37.5..."
3,"POLYGON ((-121.92192 37.53292, -121.92192 37.5..."
4,"POLYGON ((-121.92302 37.53433, -121.92302 37.5..."


In [36]:
map_4 = KeplerGl(height=1000)
map_4.add_data(data=POI_matching_lines_gdf, name = "POI matching nodes")
map_4.add_data(data=intersections, name = "Point Of Interests")
map_4.add_data(data=Aimsun_sections, name = "sections")
map_4.add_data(data=Aimsun_nodes, name = "nodes")

map_4

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md


KeplerGl(data={'POI matching nodes': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, …

In [37]:
#Set the location of all the pois of this category. maxdist is the maximum distance that will 
#later be used in find_all_nearest_pois, and maxitems - the maximum number of items that will 
#later be requested in find_all_nearest_pois

net.set_pois("All", 300, 20, intersections['Long'], intersections['Lat'])

#Find the distance to the nearest pois from each source node. 
#The bigger values in this case mean less accessibility.
access = net.nearest_pois(300, "All", num_pois=10)

In [136]:
access.head()

,1,2,3,4,5,6,7,8,9,10
id_node,,,,,,,,,,
9845,0.180000,0.237000,0.237000,0.237000,0.237000,0.237000,0.237000,0.297000,0.383,0.383000
9848,16.162001,16.219000,16.219000,16.219000,16.219000,16.219000,16.219000,16.278999,16.365,16.365000
9850,17.261999,21.434999,21.434999,21.434999,22.829000,22.886000,22.886000,22.886000,22.886,22.886000
9852,18.739000,21.351999,21.409000,21.409000,21.409000,21.409000,21.409000,21.409000,21.469,21.555000
9854,20.480000,20.480000,20.480000,24.653000,34.529999,39.132999,43.317001,44.138000,44.138,45.188999


# Accessibility visualization Measures

In [40]:
n = 1
temp_access = access[[n]]
nodes_gdf['cost'] = access[n] 
nodes_gdf.cost[:] = access[n]

In [41]:
nodes_gdf['geometry'] = [Point(xy) for xy in zip(nodes_gdf['x'], nodes_gdf['y'])]
nodes_gdf = gpd.GeoDataFrame(nodes_gdf, crs=crs)

In [42]:
nodes_gdf.head()

,id_node,x,y,cost,geometry
0,9845,-121.922491,37.495935,0.180000,POINT (-121.92249 37.49593)
1,9848,-121.921727,37.494013,16.162001,POINT (-121.92173 37.49401)
2,9850,-121.923525,37.495613,17.261999,POINT (-121.92353 37.49561)
3,9852,-121.923128,37.495265,18.739000,POINT (-121.92313 37.49526)
4,9854,-121.934705,37.505858,20.480000,POINT (-121.93470 37.50586)


In [43]:
map_5 = KeplerGl(height=600)
map_5.add_data(data=Aimsun_nodes, name = "nodes")
map_5.add_data(data=nodes_gdf, name = "accessibility heatmap")
map_5

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md


KeplerGl(data={'nodes': {'index': [9845, 9848, 9850, 9852, 9854, 9856, 9859, 9863, 9865, 9868, 9881, 9888, 989…

In [44]:
net.set_pois("POIs", 300, 20, intersections['Long'], intersections['Lat'])
# access = net.nearest_pois(300, "restaurants", num_pois=10)

# x, y = buildings.x, buildings.y
# buildings["node_ids"] = net.get_node_ids(x, y)
# net.set(node_ids, variable=buildings.square_footage, name="square_footage")
# net.set(node_ids, variable=buildings.residential_units,
#         name="residential_units")

In [45]:
net.set(intersections['nearest_node_id']) #variable=intersections['count']
aggregated = net.aggregate(60, type='sum', decay='flat') #, imp_name=None, name='tmp'
aggregated

id_node
9845     1643.0
9848     1636.0
9850     1632.0
9852     1632.0
9854     1120.0
          ...  
62955       0.0
62956       1.0
62957       0.0
62958       2.0
62959       0.0
Length: 2640, dtype: float64

In [46]:
aggregated.index = range(len(aggregated))

In [47]:
nodes_gdf.join(aggregated.to_frame())

,id_node,x,y,cost,geometry,0
0,9845,-121.922491,37.495935,0.180000,POINT (-121.92249 37.49593),1643.0
1,9848,-121.921727,37.494013,16.162001,POINT (-121.92173 37.49401),1636.0
2,9850,-121.923525,37.495613,17.261999,POINT (-121.92353 37.49561),1632.0
3,9852,-121.923128,37.495265,18.739000,POINT (-121.92313 37.49526),1632.0
4,9854,-121.934705,37.505858,20.480000,POINT (-121.93470 37.50586),1120.0
...,...,...,...,...,...,...
2635,62955,-121.944719,37.547683,300.000000,POINT (-121.94472 37.54768),0.0
2636,62956,-121.913275,37.510602,0.000000,POINT (-121.91328 37.51060),1.0
2637,62957,-121.913258,37.510625,300.000000,POINT (-121.91326 37.51062),0.0
2638,62958,-121.930401,37.537421,0.000000,POINT (-121.93040 37.53742),2.0


In [48]:
aggregated.index = range(len(aggregated))
joined = nodes_gdf.join(aggregated.to_frame())
joined = joined.rename(columns={0:'access_level'})
#joined.plot(column='access_level', cmap='OrRd', figsize=(7, 7))

In [51]:
joined.head()

,id_node,x,y,cost,geometry,access_level
0,9845,-121.922491,37.495935,0.180000,POINT (-121.92249 37.49593),1643.0
1,9848,-121.921727,37.494013,16.162001,POINT (-121.92173 37.49401),1636.0
2,9850,-121.923525,37.495613,17.261999,POINT (-121.92353 37.49561),1632.0
3,9852,-121.923128,37.495265,18.739000,POINT (-121.92313 37.49526),1632.0
4,9854,-121.934705,37.505858,20.480000,POINT (-121.93470 37.50586),1120.0


In [49]:
map_6 = KeplerGl(height=600)
map_6.add_data(data=joined, name = "aggregated accessibility heatmap")
map_6

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md


KeplerGl(data={'aggregated accessibility heatmap': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14…

## Creating accessibility isochrone with networkx

In [52]:
G = nx.Graph()

In [53]:
edge_weighted.head(5)

,id,id_node_source,id_node_target,distance,Unnamed: 0,did,oid,ent,exitTime,travelTime,delayTime
0,242,9845,9923,2100.0,233.0,27989.0,10.0,24.0,50481.9,12.94600,7.130020
1,243,9852,9848,6300.0,15.0,27989.0,1.0,16.0,50626.1,5.19072,0.000000
2,244,9850,9852,6300.0,14.0,27989.0,1.0,15.0,50620.9,1.47734,0.037734
5,248,9881,15043,700.0,9462.0,27989.0,367.0,3.0,50654.4,12.32940,8.871620
6,249,15043,9881,700.0,896850.0,27989.0,31941.0,68.0,64888.0,4.72323,1.716150


In [54]:
nodes_gdf.head(5)

,id_node,x,y,cost,geometry
0,9845,-121.922491,37.495935,0.180000,POINT (-121.92249 37.49593)
1,9848,-121.921727,37.494013,16.162001,POINT (-121.92173 37.49401)
2,9850,-121.923525,37.495613,17.261999,POINT (-121.92353 37.49561)
3,9852,-121.923128,37.495265,18.739000,POINT (-121.92313 37.49526)
4,9854,-121.934705,37.505858,20.480000,POINT (-121.93470 37.50586)


In [55]:
#load the edges and notes to networkx Graph, including the simulated results as attributes
for i in range(len(edge_weighted)):
    id_edge = edge_weighted.iloc[i]['id']
    source_node = edge_weighted.iloc[i]['id_node_source']
    target_node = edge_weighted.iloc[i]['id_node_target']
    distance = edge_weighted.iloc[i]['distance']
    travelTime = edge_weighted.iloc[i]['travelTime']
    G.add_edge(source_node, target_node, id_edge = id_edge, distance=distance, travelTime=travelTime)

In [56]:
for i in range(len(nodes_gdf)):
    id_node = nodes_gdf.iloc[i]['id_node']
    point = nodes_gdf.iloc[i]['geometry']
    cost = nodes_gdf.iloc[i]['cost']
    G.add_node(id_node, point = point, cost = cost)

In [57]:
def node_getter(index): 
    """
    retrieve the lat and lon of a node at specific index
    """
    return list(G.nodes(data=True))[index][0]
def edge_getter(index):
    """
    retrieve the starting node and endinge node of of an edge at specific index
    """
    u, v, w = list(G.edges(data=True))[index]
    return (u, v, w)


### Applying BFS on generating isochrone

In [123]:
BFS_trav = list(nx.bfs_edges(G, source=node_getter(1), depth_limit=2))
traversed = []
for t in BFS_trav:
    traversed.append(int(t[0]))
    traversed.append(int(t[1]))
traversed_2 = sorted(list(set(traversed)))

In [124]:
BFS_trav = list(nx.bfs_edges(G, source=node_getter(1), depth_limit=5))
traversed = []
for t in BFS_trav:
    traversed.append(int(t[0]))
    traversed.append(int(t[1]))
traversed_4 = sorted(list(set(traversed)))

In [125]:
node_colors = {}

for node in traversed_4:
    node_colors[node] = 2
    
for node in traversed_2:
    node_colors[node] = 1

nc = [node_colors[node] if node in node_colors else 'none' for node in G.nodes()]
#ns = [20 if node in node_colors else 0 for node in G.nodes()]
joined['color'] = nc

In [126]:
nodes_1 = {k: v for k, v in node_colors.items() if v==1}.keys()
nodes_2 = {k: v for k, v in node_colors.items() if v==2}.keys()
assert len(nodes_1) + len(nodes_2) == len(node_colors)

In [113]:
node_getter(1)

9923.0

In [114]:
# Transfering the nodes at different traversal depth to isochrone layer
def traversal_to_layer(node_set):
    layer_nodes_gdf = nodes_gdf[nodes_gdf.id_node.isin(node_set)]
    #generate a convex hull containing all the subgraph nodes 
    bounding_poly = gpd.GeoSeries(layer_nodes_gdf['geometry']).unary_union.convex_hull
    #creating a layer (geopandas df) for this bounding convex hull to be visualized in Kepler.gl
    layer = gpd.GeoDataFrame([bounding_poly], crs=crs)
    layer['geometry'] = layer[0]
    return layer[['geometry']]

In [127]:
layer_1 = traversal_to_layer(nodes_1)
layer_2 = traversal_to_layer(nodes_2)

In [128]:
map_8 = KeplerGl(height=600)
map_8.add_data(data=nodes_gdf[nodes_gdf['id_node'] == 9923], name = "center POI")
map_8.add_data(data=layer_2, name = "isochrone_2")
map_8.add_data(data=layer_1, name = "isochrone_1")#[joined['color'] == 1]
map_8

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md


KeplerGl(data={'center POI': {'index': [18], 'columns': ['id_node', 'x', 'y', 'cost', 'geometry'], 'data': [[9…

### Using networkx ego graph to plot isochrone: https://networkx.github.io/documentation/networkx-1.10/reference/generated/networkx.generators.ego.ego_graph.html

In [129]:
subgraph = nx.ego_graph(G, node_getter(1), radius=15, distance='travelTime')

In [130]:
subgraph_1 = nx.ego_graph(G, node_getter(1), radius=20, distance='travelTime')

In [131]:
def subgraph_layer(nodes_gdf, subgraph):
    #extract the Nodes dataframe for subgraph
    subgraph_nodes_gdf = nodes_gdf[nodes_gdf.id_node.isin(list(subgraph.nodes))]
    #generate a convex hull containing all the subgraph nodes 
    bounding_poly = gpd.GeoSeries(subgraph_nodes_gdf['geometry']).unary_union.convex_hull
    #creating a layer (geopandas df) for this bounding convex hull to be visualized in Kepler.gl
    layer = gpd.GeoDataFrame([bounding_poly], crs=crs)
    layer['geometry'] = layer[0]
    return layer[['geometry']]

In [132]:
layer_1 = subgraph_layer(nodes_gdf, subgraph)
layer_2 = subgraph_layer(nodes_gdf, subgraph_1)

In [135]:
map_9 = KeplerGl(height=600)
map_9.add_data(data=nodes_gdf[nodes_gdf['id_node'] == node_getter(1)], name = "all POIs")
# map_8.add_data(data=joined[joined['color'] == 1], name = "isochrone")
# map_8.add_data(data=joined[joined['color'] == 2], name = "isochrone_1")
map_9.add_data(data=layer_1, name = 'layer_1')
map_9.add_data(data=layer_2, name = 'layer_2')
map_9

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md


KeplerGl(data={'all POIs': {'index': [18], 'columns': ['id_node', 'x', 'y', 'cost', 'geometry'], 'data': [[992…

In [ ]:
nodes_gdf

## Previous experiments (might be irrelevant)

In [ ]:
# Bounding box for a small area in East Emeryville, South Berkeley and North Oakland
west, south, east, north = (-122.285535, 37.832531, -122.269571, 37.844596)

# Create a network from that bounding box
G = ox.graph_from_bbox(north, south, east, west, network_type='walk')
# Let's create n arbitrary points of interest (POI)
poi_count = 100

# this function makes those POI into Shapely points
def make_n_pois(north, south, east, west, poi_count):
    for poi in range(poi_count):
        x = (east - west) * random.random() + west
        y = (north - south) * random.random() + south
        yield Point(x, y)
        
pois = list(make_n_pois(north, south, east, west, poi_count))


# Create the plot fig and ax objects but prevent Matplotlib
# from plotting and closing out the plot operation
fig, ax = ox.plot_graph(G, fig_height=10, 
                        show=False, close=False,
                        edge_color='#777777')

# Instead, let's first update the network with these new random POI
for point in pois:
    patch = PolygonPatch(point.buffer(0.0001), fc='#ff0000', ec='k', linewidth=0, alpha=0.5, zorder=-1)
    ax.add_patch(patch)

In [ ]:
def create_nodes_df(G):
    # first make a df from the nodes
    # and pivot the results so that the 
    # individual node ids are listed as
    # row indices
    nodes_df = pd.DataFrame(G.nodes).T
    
    # preserve these indices as a column values, too
    nodes_df['id'] = nodes_df.index
    # and cast it as an integer
    nodes_df['id'] = nodes_df['id'].astype(int)
    
    return nodes_df

nodes_df = create_nodes_df(G)


In [ ]:
nodes_df = pd.DataFrame(G.nodes)

In [ ]:
nodes_df['id'] = nodes_df.index

In [ ]:
nodes_df

In [ ]:
edges_ref = {}

# move through first key (origin node)
for e1 in G.edges.keys():
    e1_dict = G.edges[e1]

    # and then get second key (destination node)
    for e2 in e1_dict.keys():
        # always use the first key here
        e2_dict = e1_dict[e2][0]

        # update the sub-dict to include
        # the origin and destination nodes
        e2_dict['st_node'] = e1
        e2_dict['en_node'] = e2

        # ugly, and unnecessary but might as
        # well name the index something useful
        name = '{}_{}'.format(e1, e2)

        # udpate the top level reference dict
        # with this new, prepared sub-dict
        edges_ref[name] = e2_dict

In [ ]:
list(G.edges)[0]

In [ ]:
G.edges[list(G.edges)[0]]

In [ ]:
G.edges[list(G.edges)[0]]

In [ ]:
for e1 in G.edges:
    e1_dict = G.edges[e1]
    print(e1_dict)

In [ ]:
# Given a graph, generate a dataframe (df)
# representing all graph edges
def create_edges_df(G):
    # First, we must move the nested objects
    # to a signle top level dictionary
    # that can be consumed by a Pandas df
    edges_ref = {}
    
    # move through first key (origin node)
    for e1 in G.edges.keys():
        e1_dict = G.edges[e1]

        # and then get second key (destination node)
        for e2 in e1_dict.keys():
            # always use the first key here
            e2_dict = e1_dict[e2][0]

            # update the sub-dict to include
            # the origin and destination nodes
            e2_dict['st_node'] = e1
            e2_dict['en_node'] = e2

            # ugly, and unnecessary but might as
            # well name the index something useful
            name = '{}_{}'.format(e1, e2)

            # udpate the top level reference dict
            # with this new, prepared sub-dict
            edges_ref[name] = e2_dict

    # let's take the resulting dict and convert it
    # to a Pandas df, and pivot it as with the nodes
    # method to get unique edges as rows
    edges_df = pd.DataFrame(edges_ref).T
    
    # udpate the edge start and stop nodes as integers
    # which is necessary for Pandana
    edges_df['st_node'] = edges_df['st_node'].astype(int)
    edges_df['en_node'] = edges_df['en_node'].astype(int)
    
    # for the purposes of this example, we are not going
    # to both with impedence along edge so they all get
    # set to the same value of 1
    edges_df['weight'] = 1
    
    return edges_df

edges_df = create_edges_df(G)

In [ ]:
#try some simple plotting of the networkx Graph
H = G.subgraph([n[0] for n in list(G.nodes(data=True))[:10]])
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
nx.draw_networkx(H, ax=ax, node_size=5,
                 font_size=6, alpha=.5,
                 width=.5)
ax.set_axis_off()

In [ ]:
plt.figure(figsize = (10,9))
m = Basemap(projection='merc',llcrnrlon=-180,llcrnrlat=10,urcrnrlon=-50,
urcrnrlat=70, lat_ts=0, resolution='l',suppress_ticks=True)

In [ ]:
#networkx graph visualization in kepler for Fremont
place_name = "Fremont, California"
graph = ox.graph_from_place(place_name, network_type='drive')
fig, ax = ox.plot_graph(graph)

nodes, edges = ox.graph_to_gdfs(graph, nodes=True, edges=True)



In [ ]:
map_3 = KeplerGl(height=600)
map_3.add_data(data=POIs_gdf, name = "Fremont Point of Interests")
map_3.add_data(data=Aimsun_nodes, name = "Fremont Aimsun nodes")
map_3.add_data(data=Aimsun_sections, name = "Fremont edges")
map_3

In [ ]:
from geopy.distance import geodesic 
  
# Loading the lat-long data for Kolkata & Delhi 

# Print the distance calculated in km 



#for (u, v, d) in G.edges(data=True):
    
    

For each selected starting POI, we first find and link it to the closest Aimsun network node. 
Then, by running the BFS, we find the range of reachable spots (accessibility area) from this traversal and count the amount of POIs of different categories reachable within the area. 
And develop an algorithm to calculate the accessibility score


## Abandoned accessibility isochrone (keep for potential future reference)

In [ ]:
import geopy.distance
#geopy.distance.distance(coord, coord1).m
# Trip time in seconds
trip_times = [10, 20, 30, 40, 50]
#iso_colors = ox.get_colors(n=len(trip_times), cmap='Reds', start=0.3, return_hex=True)
iso_colors = [1, 2, 3, 4, 5]

G.edges[(9845, 9923.0)]

# 2 - color the nodes according to isochrone then plot the street network
node_colors = {}
subgraphs = []
for trip_time, color in zip(sorted(trip_times, reverse=True), iso_colors):
    subgraph = nx.ego_graph(G, intersections.iloc[684]['nearest_node_id'], radius=trip_time, distance='travelTime')
    for node in subgraph.nodes():
        node_colors[node] = color
    subgraphs.append(subgraph)
    

#fig, ax = ox.plot_graph(G, fig_height=8, node_color=nc, node_size=ns, node_alpha=0.8, node_zorder=2)


nc = [node_colors[node] if node in node_colors else 'none' for node in G.nodes()]
ns = [20 if node in node_colors else 0 for node in G.nodes()]
joined['color'] = nc

map_7 = KeplerGl(height=600)
map_7.add_data(data=intersections[intersections['nearest_node_id'] == 48341], name = "all POIs")
map_7.add_data(data=joined[joined['color'] == 1], name = "isochrone")
map_7.add_data(data=joined[joined['color'] == 2], name = "isochrone_1")
map_7.add_data(data=joined[joined['color'] == 3], name = "isochrone_2")
map_7.add_data(data=joined[joined['color'] == 4], name = "isochrone_3")
map_7.add_data(data=joined[joined['color'] == 5], name = "isochrone_4")
map_7.add_data(data=joined[joined['color'] == 'none'], name = "isochrone_5")
map_7

In [ ]:
# import geopandas as gpd
# import numpy as np
# import pandas as pd

# from scipy.spatial import cKDTree
# from shapely.geometry import Point

# gpd1 = gpd.GeoDataFrame([['John', 1, Point(1, 1)], ['Smith', 1, Point(2, 2)],
#                          ['Soap', 1, Point(0, 2)]],
#                         columns=['Name', 'ID', 'geometry'])
# gpd2 = gpd.GeoDataFrame([['Work', Point(0, 1.1)], ['Shops', Point(2.5, 2)],
#                          ['Home', Point(1, 1.1)]],
#                         columns=['Place', 'geometry'])

# def ckdnearest(gdA, gdB):
#     nA = np.array(list(zip(gdA.geometry.x, gdA.geometry.y)) )
#     nB = np.array(list(zip(gdB.geometry.x, gdB.geometry.y)) )
#     btree = cKDTree(nB)
#     dist, idx = btree.query(nA, k=1)
#     gdf = pd.concat(
#         [gdA.reset_index(drop=True), gdB.loc[idx, gdB.columns != 'geometry'].reset_index(drop=True),
#          pd.Series(dist, name='dist')], axis=1)
#     return gdf

# ckdnearest(gpd1, gpd2)